In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf, pandas as pd

In [2]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
#df['day'] = df.ts % (60*60*24*7)
#df['aid2'] = df.aid
print( df.shape )
df.head()

(6928123, 4)


,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


In [3]:
user_features = df.groupby('session').agg({'ts':'max'})
user_features.columns = ['ts_max7']

In [4]:
user_features.head()

,ts_max7
session,
13191107,1661829345
13419212,1662113401
13711008,1661981622
13750431,1662017929
13775993,1662026758


In [5]:
user_features2 = df.groupby('session').agg({'ts':'min'})
user_features2.columns = ['ts_min7']

In [6]:
user_features2.head()

,ts_min7
session,
13191107,1661829277
13419212,1661888079
13711008,1661981622
13750431,1662017929
13775993,1662026687


In [7]:
user_features = cudf.concat([user_features,user_features2],axis=1)

In [8]:
user_features['ts_diff7'] = user_features.ts_max7 - user_features.ts_min7

In [9]:
for c in ['ts_max7','ts_min7']:
    user_features[c] -= mn

In [10]:
user_features.head()

,ts_max7,ts_min7,ts_diff7
session,,,
12899779,0,0,0
12899780,155,0,155
12899781,336160,0,336160
12899782,79953,0,79953
12899783,317140,0,317140


In [11]:
user_features = user_features.astype('int32')

In [12]:
user_features.to_parquet('../../data/item_user_features/user7_LB.pqt')